<h1> Step 2: Get historic weather data </h1>

In [1]:
# Import packages
# For data frames
import pandas as pd
import numpy as np
# For weather forcast
from darksky import forecast
# For dates
from datetime import datetime as dt
from datetime import timedelta
# For holidays
import holidays

In [2]:
# Set parameters for weather extraction
with open('dark_sky_apikey.txt', 'r') as myfile:
    apikey = myfile.readlines()
phila = [39.9526, -75.1652]
PHILA = apikey[0], phila[0], phila[1]
us_holidays = holidays.UnitedStates()

In [17]:
# Set dates for weather extraction
which_year = 2016
start_date = str(which_year-1) +"-12-31"
if which_year == 2018:
    stop_date = str(which_year) + "-10-01"
else: 
    stop_date = str(which_year+1) + "-01-01"
start = dt.strptime(start_date, "%Y-%m-%d")
stop = dt.strptime(stop_date, "%Y-%m-%d")
# Quick check of how many days are requested
print(stop-start)

367 days, 0:00:00


In [4]:
def extract_weather_info(start, stop, us_holidays, weatherObject):
    # Initialize the structure that we'll populate with data
    d = []
    # First, loop through dates and get weather data
    # Then loop through the hours of the day and extract hourly weather data 
    while start < stop:
        start = start + timedelta(days=1)
        start_time = start.isoformat()
        pp = forecast(*weatherObject, time = start_time)

        # check the length of hours structure
        nHours = 24
        tempHours = len(pp.hourly)
        if tempHours < nHours: 
            nHours = len(pp.hourly)

        for i in range(0,nHours): 
            # 1 precipitation intensity
            pInt = None
            try: 
                pInt = pp.hourly[i].precipIntensity
            except(AttributeError):
                pass

            # 2 preciptiation Type
            pType = None
            try: 
                pType = pp.hourly[i].precipType
            except(AttributeError):
                pass

            # 3 precipitation Probability
            pProb = None
            try: 
                pProb = pp.hourly[i].precipProbability
            except(AttributeError):
                pass 

            # 4 temperature
            temp = None
            try: 
                temp = pp.hourly[i].temperature
            except(AttributeError):
                pass 

            # 5 apparent temperature
            appTemp = None
            try: 
                appTemp = pp.hourly[i].apparentTemperature
            except(AttributeError):
                pass 

            # 6 humidity
            humid = None
            try: 
                humid = pp.hourly[i].humidity
            except(AttributeError):
                pass  

            # 7 pressure
            press = None
            try: 
                press = pp.hourly[i].pressure
            except(AttributeError):
                pass    

            # 8 wind speed
            wSpeed = None
            try: 
                wSpeed = pp.hourly[i].windSpeed
            except(AttributeError):
                pass      

            # 9 wind bearing
            wBearing = None
            try: 
                wBearing = pp.hourly[i].windBearing
            except(AttributeError):
                pass      

            # 10 visibiilty
            visib = None
            try: 
                visib = pp.hourly[i].visibility
            except(AttributeError):
                pass 

            d.append({'date_time': start_time,
                'day': start.day,
                'month': start.month,
                'hour': start + timedelta(hours = i),
                'weekday': start.weekday(), 
                'is_holiday': start in us_holidays,
                'which_holiday':  holidays.US(years = start.year).get(start),     
                'precipIntensity': pInt, 
                'precipProbability': pProb,
                'precipType': pType, 
                'temperature': temp,  
                'apparentTemperature': appTemp,  
                'humidity': humid,  
                'pressure': press,  
                'windSpeed': wSpeed,  
                'windBearing': wBearing, 
                'visibility': visib})
            # variables that are there but I am not using them: 
            # uv_index, icon, cloudCover, summary, dewPoint
    df = pd.DataFrame(d)
    return df

In [5]:
# Get the dataframe and save it
weather_df = extract_weather_info(start, stop, us_holidays, PHILA)
weather_filename = "Weather" + str(which_year) + ".csv"
weather_df.to_csv(weather_filename)